# Build the logbook for Planetary Nebula of September 2021

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : September 22th 2021

Works directory per directory (day by day)

CCIN2P3:

- works with **kernels anaconda3_auxtel** (with libradtran) 

In [103]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookscc/ana_summer2021',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.egg',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages/pyarrow-4.0.1-py3.8-linux-x86_64.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/coveralls-3.0.1-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/astroquery-0.4.2.dev0-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs

In [104]:
# reconstructed / good /xpos /ypos
info={\
      'holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000166_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000176_quickLookExp.fits':(1,1,300., 1750.),\
       'holo4_003_empty_PNG321.0+03.9_20210909_000177_quickLookExp.fits':(1,1,300., 1750.)     
     }

# Imports

In [105]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [106]:
from astropy.io import fits

In [107]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [108]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [109]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [110]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'image',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [111]:
target_name_selected="PNG321.0+03.9"

In [112]:
dir_logbooks="logbooks"

In [113]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [114]:
FLAG_HOLO=True
home=os.getenv("HOME")

In [115]:
if FLAG_HOLO:
    image_path="holo"
    version="v2.0"
    logbook_csv=os.path.join(dir_logbooks,"auxtelholologbook_PNG_2021_09_09_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelholologbook_PNG_2021_09_09_" + version + ".xlsx")
else:
    image_path="ronchi170lpmm"
    version="v2.0"
    logbook_csv=os.path.join(dir_logbooks,"auxtelronchilogbook_PNG_2021_09_09_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelronchilogbook_PNG_2021_09_09_" + version + ".xlsx")

In [116]:
#dir_images="/Users/dagoret/DATA/AuxTelData2021/holo/FlipCleans"
# laptop
#dir_images=os.path.join(home,"DATA/AuxTelData2021/")+image_path+"/FlipCleans"
# CC
#dir_images="/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans"
dir_images="/sps/lsst/groups/auxtel/data/2021/holo/quickLookExp/2021-09-09"

In [117]:
all_files=os.listdir(dir_images)

In [118]:
all_files

['holo4_003_empty_HD2811_20210909_000591_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000584_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000572_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000595_quickLookExp.fits',
 'holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000586_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000588_quickLookExp.fits',
 'holo4_003_empty_HD2811_20210909_000592_quickLookExp.fits',
 'holo4_003_empty_HD2811_2021

In [119]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [120]:
all_selected_files=np.array(all_selected_files)

In [121]:
print(all_selected_files)

['holo4_003_empty_HD2811_20210909_000591_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000584_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000572_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000595_quickLookExp.fits'
 'holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000586_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000588_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000592_quickLookExp.fits'
 'holo4_003_empty_HD2811_20210909_000597_qui

In [122]:
re.findall('^holo4_003_.*_(.*)[.]fits$','holo4_003_empty_HD115169_20210608_000291.fits')

['000291']

### Sort files

In [123]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('^holo4_003_.*_(.*)_quickLookExp[.]fits$',filename)
    
    return m[0]

In [124]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['000591', '000165', '000167', '000168', '000174', '000584',
       '000171', '000169', '000572', '000170', '000595', '000175',
       '000586', '000588', '000592', '000597', '000596', '000176',
       '000579', '000576', '000177', '000569', '000585', '000570',
       '000568', '000598', '000590', '000573', '000593', '000578',
       '000574', '000589', '000594', '000581', '000583', '000571',
       '000599', '000577', '000582', '000567', '000580', '000587',
       '000166', '000575'], dtype='<U6')

In [125]:
indexes=np.argsort(all_tags)

In [126]:
sorted_files=all_selected_files[indexes]

In [127]:
sorted_files

array(['holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000166_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000176_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000177_quickLookExp.fits',
       'holo4_003_empty_HD2811_20210909_000567_quickLookExp.fits',
       'holo4_003_empty_HD2811_20210909_000568_quickLookExp.fits',
       'holo4_003_empty_HD2811_20210909_000569_quick

In [128]:
N=len(sorted_files)

### Filter bad files

In [129]:
idx=0
indexes_bad=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0['OBJECT'],"\t",filename)
    if hdr0['OBJECT'] == '35 Tau' or hdr0['OBJECT'] == 'Park position':
        print('bad file ', filename)
        indexes_bad.append(idx)
        
    idx+=1

In [130]:
indexes_bad

[]

### Remove bad files

In [131]:
sorted_files=np.delete(sorted_files,indexes_bad)

### Check for target

In [132]:
re.findall('^holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*_quickLookExp[.]fits$','holo4_003_RG610_HD120144_20210706_000098.fits')

[]

In [133]:
def file_target(filename):
    m=re.findall('^holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*_quickLookExp[.]fits$',filename)
    if len(m)>0:
        return m
    elif re.search('NGC4755',filename):
        return ['NGC4755']
    else:
        return []

In [134]:
for filename in sorted_files:
    print(file_target(filename),"\t",filename)

['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000166_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000176_quickLookExp.fits
['PNG321.0+03.9'] 	 holo4_003_empty_PNG321.0+03.9_20210909_000177_quickLookExp.fits
['HD2811'] 	 holo4_003_empty_HD2811_20210909_000567_quickLookExp.fits
['HD28

In [135]:
idx=0
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0,"\t",filename)
        
    idx+=1

# Build the logbook from the headers

In [136]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=file_target(filename)[0]
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [137]:
df=pd.DataFrame()

In [138]:
len(all_airmass)

44

In [139]:
N=len(all_indexes)

## Fill with container info

In [140]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [141]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [142]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [143]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [144]:
pd.set_option('display.max_rows', None)

In [145]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2021-09-09T23:53:55.100,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000165_...,165,PNG321.0+03.9,empty_1,holo4_003,1.380357,10.0,...,9.2,16.0,4.0,300.0,1750.0,NaN,NaN,,1.0,1.0
1,1,2021-09-09T23:54:18.095,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000166_...,166,PNG321.0+03.9,empty_1,holo4_003,1.381613,10.0,...,9.1,16.0,3.6,300.0,1750.0,NaN,NaN,,1.0,1.0
2,2,2021-09-09T23:54:41.122,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000167_...,167,PNG321.0+03.9,empty_1,holo4_003,1.382874,10.0,...,9.1,16.0,3.6,300.0,1750.0,NaN,NaN,,1.0,1.0
3,3,2021-09-09T23:55:04.180,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000168_...,168,PNG321.0+03.9,empty_1,holo4_003,1.384148,30.0,...,9.1,16.0,3.6,300.0,1750.0,NaN,NaN,,1.0,1.0
4,4,2021-09-09T23:55:47.196,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000169_...,169,PNG321.0+03.9,empty_1,holo4_003,1.386523,30.0,...,9.0,17.0,3.1,300.0,1750.0,NaN,NaN,,1.0,1.0
5,5,2021-09-09T23:56:30.224,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000170_...,170,PNG321.0+03.9,empty_1,holo4_003,1.388908,30.0,...,9.0,16.0,3.9,300.0,1750.0,NaN,NaN,,1.0,1.0
6,6,2021-09-09T23:57:13.142,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000171_...,171,PNG321.0+03.9,empty_1,holo4_003,1.391300,60.0,...,9.0,16.0,3.9,300.0,1750.0,NaN,NaN,,1.0,1.0
7,7,2021-09-10T00:04:18.230,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000174_...,174,PNG321.0+03.9,empty_1,holo4_003,1.415803,0.0,...,8.9,15.0,3.7,300.0,1750.0,NaN,NaN,,1.0,1.0
8,8,2021-09-10T00:06:01.239,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000175_...,175,PNG321.0+03.9,empty_1,holo4_003,1.421962,60.0,...,8.9,15.0,3.8,300.0,1750.0,NaN,NaN,,1.0,1.0
9,9,2021-09-10T00:07:14.156,/sps/lsst/groups/auxtel/data/2021/holo/quickLo...,holo4_003_empty_PNG321.0+03.9_20210909_000176_...,176,PNG321.0+03.9,empty_1,holo4_003,1.426365,60.0,...,8.9,15.0,4.4,300.0,1750.0,NaN,NaN,,1.0,1.0


# Remove non Planetary Nebula rows

In [146]:
df=df.loc[df['object'] == target_name_selected]

In [147]:
df['file'].values

array(['holo4_003_empty_PNG321.0+03.9_20210909_000165_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000166_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000167_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000168_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000169_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000170_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000171_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000174_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000175_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000176_quickLookExp.fits',
       'holo4_003_empty_PNG321.0+03.9_20210909_000177_quickLookExp.fits'],
      dtype=object)

# Make logbook


In [148]:
df.to_excel(logbook_xlsx)
df.to_csv(logbook_csv)

In [149]:
logbook_csv

'logbooks/auxtelholologbook_PNG_2021_09_09_v2.0.csv'